In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from Multiband import Multiband
from MPbandTool import MPbandTool
import pandas as pd
from ga_platoon import max_dt

['f:\\workspace\\jupyter\\论文\\graduation thesis\\Case Analysis', 'd:\\anaconda\\envs\\cplex\\python37.zip', 'd:\\anaconda\\envs\\cplex\\DLLs', 'd:\\anaconda\\envs\\cplex\\lib', 'd:\\anaconda\\envs\\cplex', '', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\fonttools-4.36.0-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\cvxopt-1.3.0-py3.7-win-amd64.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\pytz-2022.2.1-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\Pythonwin', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\0\\.ipython', 'f:\\wor

In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 4, 10
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001   
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g 
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

p=[0,0,0,0,1,0,0,0,1,0,1,0,0]

In [4]:
mb=Multiband(phase,[80,180],straight_get,p,d,lower,upper,speedVar[0], speedVar[1],vol,px)
mb._solve()

13 13
conflict(s): 0
status  = integer optimal solution
time    = 0.016 s.
problem = MILP
gap     = 0%

object value: 2004.6129999999998


In [5]:
a=mb.get_dataframe()
a

,cross_number,distance,sg1,sg2,b1,b2,offset,p,t1,t2,z,u1,u2,w1,car_t1,w2,car_t2
0,1.0,800.0,0.282,0.330,0.282,0.330,0.000000,0.0,80.000000,87.709637,167.709637,0.000000,0.000000,0.1410,23.506059,0.165000,74.801208
1,2.0,520.0,0.426,0.368,0.187,0.330,55.849812,0.0,130.000000,56.576971,167.709637,0.000000,0.000000,0.2850,103.553559,0.165000,154.801208
2,3.0,500.0,0.187,0.200,0.187,0.200,50.256571,0.0,125.000000,68.704631,167.709637,0.000000,0.000000,0.0935,65.843922,0.100000,98.289237
3,4.0,490.0,0.277,0.277,0.277,0.200,0.000000,0.0,49.000000,49.000000,167.709637,0.000000,0.000000,0.1385,23.089285,0.177000,29.584606
4,5.0,370.0,0.255,0.255,0.255,0.255,0.000000,1.0,45.110722,84.521917,109.266320,0.696828,0.757328,0.1275,13.803956,0.127500,13.803956
5,6.0,254.0,0.650,0.650,0.243,0.255,1.950525,0.0,63.500000,25.400000,109.266320,0.000000,0.000000,0.5225,58.920677,0.336108,38.548358
6,7.0,585.0,0.243,0.243,0.165,0.243,0.000000,0.0,105.004933,113.527706,109.266320,0.000000,0.000000,0.1215,13.193358,0.121500,13.154358
7,8.0,1020.0,0.165,0.165,0.165,0.165,0.000000,0.0,102.000000,102.000000,109.266320,0.000000,0.000000,0.0825,8.931971,0.082500,8.931971
8,9.0,409.0,0.298,0.298,0.298,0.298,80.000000,1.0,83.240000,80.000000,80.000000,0.133001,0.000001,0.1490,91.771000,0.149000,91.771000
9,10.0,547.0,0.379,0.379,0.379,0.298,0.000000,0.0,79.999920,72.319920,80.000000,0.000000,0.000000,0.1895,14.970500,0.149000,11.771000


In [15]:
a[["b"+str(i+1) for i in range(2)]].mean()

b1    0.262846
b2    0.287923
dtype: float64

In [6]:
mpb=MPbandTool(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
            be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],p,a.offset/a.z,
            1/a.z,np.array([a.t1/a.z,a.t2/a.z]))
mpb._solve()

conflict(s): 0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 0.0000.
Tried aggregator 3 times.
MIP Presolve eliminated 793 rows and 287 columns.
MIP Presolve modified 902 coefficients.
Aggregator did 62 substitutions.
Reduced MIP has 448 rows, 219 columns, and 1132 nonzeros.
Reduced MIP has 27 binaries, 37 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.98 ticks)
Probing fixed 0 vars, tightened 52 bounds.
Probing time = 0.00 sec. (0.11 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 60 rows and 50 columns.
MIP Presolve modified 189 coefficients.
Reduced MIP has 388 rows, 169 columns, and 990 nonzeros.
Reduced MIP has 16 binaries, 37 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.53 ticks)
Probing time = 0.00 sec. (0.08 ticks)
Tried ag

In [7]:
# colors=[
#             {"color":"gray" , "hatch": None, "fill": True},
#             {"color":"gray", "hatch":"||||" , "fill": True},
#             {"color": "gray", "hatch":"////" , "fill": True},
#             {"color":"white", "hatch": "****", "fill": False},
#             {"color": "white", "hatch": None, "fill": False},
#             {"color": "white", "hatch": "////", "fill": False},
#             {"color": "white", "hatch": "||||", "fill": False},
#             {"color": "white", "hatch": "\\\\\\\\", "fill": False},
#             {"color": "white", "hatch": "xxxx", "fill": False},
#             {"color": "white", "hatch": "....", "fill": False},
# ]
# linestyles=[
#     {"linestyle":"-" },
#     {"linestyle":(5,(10,3))},
#     {"linestyle":(0, (5, 3)) },
#     {"linestyle":":" },
#     {"linestyle":(0, (3, 1, 1, 1)) },
#     {"linestyle": (0, (5, 10)) },
# ]
# legends=["1->15", "1->6", "8->15", "15->2", "14->9", "7->1"]
# mb.draw_car_bound("./时距图",colors,legends,2,linestyles)

In [10]:
b=mpb.get_dataframe()
b

,cross_number,distance,b1,b2,b3,b4,b5,b6,offset,p,...,u3,u4,u5,u6,bb1,bb2,dw1,dw2,tb1,tb2
0,1.0,800.0,0.282,0.282,0.000,0.326576,0.000,0.330,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.282,0.330,30.0,30.0,230.00,230.00
1,2.0,520.0,0.187,0.187,0.000,0.330000,0.000,0.330,55.849812,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.187,0.330,15.0,15.0,52.00,52.00
2,3.0,500.0,0.187,0.187,0.000,0.200000,0.000,0.200,50.256571,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.187,0.200,30.0,30.0,155.00,155.00
3,4.0,490.0,0.277,0.277,0.000,0.200000,0.000,0.200,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.277,0.200,15.0,15.0,49.00,49.00
4,5.0,370.0,0.255,0.000,0.255,0.255000,0.000,0.255,0.000000,1.0,...,8.353284e-01,7.573284e-01,1.000000e-08,7.573284e-01,0.255,0.255,30.0,30.0,122.50,122.50
5,6.0,254.0,0.243,0.000,0.243,0.255000,0.000,0.255,1.950525,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.243,0.255,15.0,15.0,25.40,25.40
6,7.0,585.0,0.165,0.000,0.165,0.183404,0.000,0.243,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.165,0.243,30.0,30.0,176.25,176.25
7,8.0,1020.0,0.165,0.000,0.165,0.165000,0.000,0.165,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.165,0.165,30.0,30.0,285.00,285.00
8,9.0,409.0,0.298,0.000,0.298,0.298000,0.298,0.000,80.000000,1.0,...,1.330010e-01,1.000001e+00,8.250100e-02,1.000000e-08,0.298,0.298,30.0,30.0,132.25,132.25
9,10.0,547.0,0.379,0.000,0.379,0.298000,0.298,0.000,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.379,0.298,30.0,30.0,166.75,166.75


In [12]:
b[["b"+str(i+1) for i in range(6)]].mean(axis=0)

b1    0.262846
b2    0.071769
b3    0.191077
b4    0.283075
b5    0.135769
b6    0.152154
dtype: float64